# Exercises: Chapter 4

In [804]:
import pandas as pd
import numpy as np

#### 1. With the `earthquakes.csv` file, select all the earthquakes in Japan with a magnitude of 4.9 or greater using the mb magnitude type.

In [805]:
!ls exercises/

README.md         covid19_cases.csv earthquakes.csv   faang.csv


In [806]:
!head -5 exercises/earthquakes.csv

mag,magType,time,place,tsunami,parsed_place
1.35,ml,1539475168010,"9km NE of Aguanga, CA",0,California
1.29,ml,1539475129610,"9km NE of Aguanga, CA",0,California
3.42,ml,1539475062610,"8km NE of Aguanga, CA",0,California
0.44,ml,1539474978070,"9km NE of Aguanga, CA",0,California


In [807]:
quakes = pd.read_csv('exercises/earthquakes.csv')

In [808]:
quakes.head()

,mag,magType,time,place,tsunami,parsed_place
0,1.35,ml,1539475168010,"9km NE of Aguanga, CA",0,California
1,1.29,ml,1539475129610,"9km NE of Aguanga, CA",0,California
2,3.42,ml,1539475062610,"8km NE of Aguanga, CA",0,California
3,0.44,ml,1539474978070,"9km NE of Aguanga, CA",0,California
4,2.16,md,1539474716050,"10km NW of Avenal, CA",0,California


In [809]:
quakes.dtypes

mag             float64
magType          object
time              int64
place            object
tsunami           int64
parsed_place     object
dtype: object

In [810]:
quakes.shape

(9332, 6)

In [811]:
quakes.loc[lambda x: (x.mag >= 4.9) & (x.magType == 'mb') & (x.parsed_place == 'Japan') ]

,mag,magType,time,place,tsunami,parsed_place
1563,4.9,mb,1538977532250,"293km ESE of Iwo Jima, Japan",0,Japan
2576,5.4,mb,1538697528010,"37km E of Tomakomai, Japan",0,Japan
3072,4.9,mb,1538579732490,"15km ENE of Hasaki, Japan",0,Japan
3632,4.9,mb,1538450871260,"53km ESE of Hitachi, Japan",0,Japan


In [812]:
quakes[quakes.mag.ge(4.9) & (quakes.magType.eq('mb')) & (quakes.parsed_place.eq('Japan'))]

,mag,magType,time,place,tsunami,parsed_place
1563,4.9,mb,1538977532250,"293km ESE of Iwo Jima, Japan",0,Japan
2576,5.4,mb,1538697528010,"37km E of Tomakomai, Japan",0,Japan
3072,4.9,mb,1538579732490,"15km ENE of Hasaki, Japan",0,Japan
3632,4.9,mb,1538450871260,"53km ESE of Hitachi, Japan",0,Japan


In [813]:
quakes[quakes.mag.ge(4.9) & quakes.magType.eq('mb')]

,mag,magType,time,place,tsunami,parsed_place
227,5.2,mb,1539389603790,"15km WSW of Pisco, Peru",0,Peru
229,4.9,mb,1539389546300,"193km N of Qulansiyah, Yemen",0,Yemen
248,4.9,mb,1539382925190,"151km S of Severo-Kuril'sk, Russia",0,Russia
258,5.1,mb,1539380306940,"236km NNW of Kuril'sk, Russia",0,Russia
391,5.1,mb,1539337221080,Pacific-Antarctic Ridge,0,Pacific-Antarctic Ridge
...,...,...,...,...,...,...
9154,4.9,mb,1537268270010,Southwest Indian Ridge,0,Southwest Indian Ridge
9175,5.2,mb,1537262729590,"126km N of Dili, East Timor",1,East Timor
9176,5.2,mb,1537262656830,"90km S of Raoul Island, New Zealand",0,New Zealand
9213,5.1,mb,1537255481060,South of Tonga,0,Tonga


In [814]:
quakes.query('(mag >= 4.9) and (magType == "mb") and (parsed_place == "Japan")')

,mag,magType,time,place,tsunami,parsed_place
1563,4.9,mb,1538977532250,"293km ESE of Iwo Jima, Japan",0,Japan
2576,5.4,mb,1538697528010,"37km E of Tomakomai, Japan",0,Japan
3072,4.9,mb,1538579732490,"15km ENE of Hasaki, Japan",0,Japan
3632,4.9,mb,1538450871260,"53km ESE of Hitachi, Japan",0,Japan


#### 2. Create bins for each full number of earthquake magnitude (for instance, the first bin is (0, 1], the second is (1, 2], and so on) with the ml magnitude type and count how many are in each bin.

In [815]:
##??pd.cut

In [816]:
ml_mags = (
    quakes
    .query('magType == "ml"')['mag']
    .to_frame()
)

# find min and max values
# agg might not work on a series
ml_mags.agg({'mag':['min', 'max']}).T

,min,max
mag,-1.26,5.1


In [817]:
ml_mag_bins = pd.cut(ml_mags.squeeze(), bins = range(-2,7))
ml_mag_bins

0       (1, 2]
1       (1, 2]
2       (3, 4]
3       (0, 1]
6       (1, 2]
         ...  
9325    (0, 1]
9326    (1, 2]
9328    (0, 1]
9330    (1, 2]
9331    (0, 1]
Name: mag, Length: 6803, dtype: category
Categories (8, interval[int64, right]): [(-2, -1] < (-1, 0] < (0, 1] < (1, 2] < (2, 3] < (3, 4] < (4, 5] < (5, 6]]

In [818]:
ml_mag_bins.value_counts()

(1, 2]      3105
(0, 1]      2207
(2, 3]       862
(-1, 0]      491
(3, 4]       122
(-2, -1]      13
(4, 5]         2
(5, 6]         1
Name: mag, dtype: int64

In [819]:
ml_mag_bins.value_counts(normalize=True)

(1, 2]      0.456416
(0, 1]      0.324416
(2, 3]      0.126709
(-1, 0]     0.072174
(3, 4]      0.017933
(-2, -1]    0.001911
(4, 5]      0.000294
(5, 6]      0.000147
Name: mag, dtype: float64

#### 3. Using the faang.csv file, group by the ticker and resample to monthly frequency. Make the following aggregations:

a) Mean of opening price

b) Maximum of the high price

c) Minimum of the low price

d) Mean of the closing price

e) Sum of the volume traded

In [820]:
!head -5 exercises/faang.csv

ticker,date,high,low,open,close,volume
FB,2018-01-02,181.5800018310547,177.5500030517578,177.67999267578125,181.4199981689453,18151900.0
FB,2018-01-03,184.77999877929688,181.3300018310547,181.8800048828125,184.6699981689453,16886600.0
FB,2018-01-04,186.2100067138672,184.1000061035156,184.8999938964844,184.3300018310547,13880900.0
FB,2018-01-05,186.8999938964844,184.92999267578125,185.58999633789062,186.8500061035156,13574500.0


In [821]:
faang = pd.read_csv('exercises/faang.csv')
faang.head()

,ticker,date,high,low,open,close,volume
0,FB,2018-01-02,181.580002,177.550003,177.679993,181.419998,18151900.0
1,FB,2018-01-03,184.779999,181.330002,181.880005,184.669998,16886600.0
2,FB,2018-01-04,186.210007,184.100006,184.899994,184.330002,13880900.0
3,FB,2018-01-05,186.899994,184.929993,185.589996,186.850006,13574500.0
4,FB,2018-01-08,188.899994,186.330002,187.199997,188.279999,17994700.0


In [822]:
faang.shape

(1255, 7)

In [823]:
faang.info() # to group by month, want to make date the index as a datetime

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  1255 non-null   object 
 1   date    1255 non-null   object 
 2   high    1255 non-null   float64
 3   low     1255 non-null   float64
 4   open    1255 non-null   float64
 5   close   1255 non-null   float64
 6   volume  1255 non-null   float64
dtypes: float64(5), object(2)
memory usage: 68.8+ KB


In [824]:
faang = pd.read_csv('exercises/faang.csv', index_col='date', parse_dates=True)
faang.head()

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-04,FB,186.210007,184.100006,184.899994,184.330002,13880900.0
2018-01-05,FB,186.899994,184.929993,185.589996,186.850006,13574500.0
2018-01-08,FB,188.899994,186.330002,187.199997,188.279999,17994700.0


In [825]:
faang.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1255 entries, 2018-01-02 to 2018-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  1255 non-null   object 
 1   high    1255 non-null   float64
 2   low     1255 non-null   float64
 3   open    1255 non-null   float64
 4   close   1255 non-null   float64
 5   volume  1255 non-null   float64
dtypes: float64(5), object(1)
memory usage: 68.6+ KB


In [826]:
( faang
 .groupby('ticker')
 .resample('M')
 .agg({
     'open':'mean',
     'high':'max',
     'low':'min',
     'close':'mean',
     'volume':'sum'})
)

open         high          low        close  \
ticker date                                                             
AAPL   2018-01-31    43.505357    45.025002    41.174999    43.501309   
       2018-02-28    41.819079    45.154999    37.560001    41.909737   
       2018-03-31    43.761786    45.875000    41.235001    43.624048   
       2018-04-30    42.441310    44.735001    40.157501    42.458572   
       2018-05-31    46.239091    47.592499    41.317501    46.384205   
       2018-06-30    47.180119    48.549999    45.182499    47.155357   
       2018-07-31    47.549048    48.990002    45.855000    47.577857   
       2018-08-31    53.121739    57.217499    49.327499    53.336522   
       2018-09-30    55.582763    57.417500    53.825001    55.518421   
       2018-10-31    55.300000    58.367500    51.522499    55.211413   
       2018-11-30    47.954881    55.590000    42.564999    47.808929   
       2018-12-31    41.310789    46.235001    36.647499    41.066579   
AMZN   2018-01-31  1301.377151  1472.579956  1170.510010  1309.010946   
       2018-02-28  1447.113159  1528.699951  1265.930054  1442.363146   
       2018-03-31  1542.160464  1617.540039  1365.199951  1540.367629   
       2018-04-30  1475.841902  1638.099976  1352.880005  1468.220471   
       2018-05-31  1590.474543  1635.000000  1546.020020  1594.903637   
       2018-06-30  1699.088582  1763.099976  1635.089966  1698.823812   
       2018-07-31  1786.305716  1880.050049  1678.060059  1784.649042   
       2018-08-31  1891.957833  2025.569946  1776.020020  1897.851308   
       2018-09-30  1969.239476  2050.500000  1865.000000  1966.077900   
       2018-10-31  1799.630865  2033.189941  1476.359985  1782.058265   
       2018-11-30  1622.323806  1784.000000  1420.000000  1625.483823   
       2018-12-31  1572.922100  1778.339966  1307.000000  1559.443154   
FB     2018-01-31   184.584284   190.660004   175.800003   184.962856   
       2018-02-28   180.721578   195.320007   167.179993   180.269473   
       2018-03-31   173.449524   186.100006   149.020004   173.489522   
       2018-04-30   164.163332   177.100006   150.509995   163.810476   
       2018-05-31   181.910909   192.720001   170.229996   182.930000   
       2018-06-30   194.974763   203.550003   186.429993   195.267620   
       2018-07-31   199.332381   218.619995   166.559998   199.967142   
       2018-08-31   177.598695   188.300003   170.270004   177.492172   
       2018-09-30   164.233158   173.889999   158.869995   164.377368   
       2018-10-31   154.873479   165.880005   139.029999   154.187826   
       2018-11-30   141.762857   154.130005   126.849998   141.635715   
       2018-12-31   137.529475   147.190002   123.019997   137.161052   
GOOG   2018-01-31  1127.200945  1186.890015  1045.229980  1130.770467   
       2018-02-28  1088.629472  1174.000000   992.559998  1088.206839   
       2018-03-31  1096.108085  1177.050049   980.640015  1091.490479   
       2018-04-30  1038.415237  1094.165039   990.369995  1035.696187   
       2018-05-31  1064.021376  1110.750000  1006.289978  1069.275901   
       2018-06-30  1136.396182  1186.286011  1096.010010  1137.626668   
       2018-07-31  1183.464280  1273.890015  1093.800049  1187.590472   
       2018-08-31  1226.156951  1256.500000  1188.239990  1225.671732   
       2018-09-30  1176.878424  1212.989990  1146.910034  1175.808934   
       2018-10-31  1116.082172  1209.959961   995.830017  1110.940411   
       2018-11-30  1054.971424  1095.569946   996.020020  1056.162394   
       2018-12-31  1042.619998  1124.650024   970.109985  1037.420519   
NFLX   2018-01-31   231.269525   286.809998   195.419998   232.908096   
       2018-02-28   270.873158   297.359985   236.110001   271.443683   
       2018-03-31   312.712859   333.980011   275.899994   312.228097   
       2018-04-30   309.129524   338.820007   271.220001   307.466192   
       2018-05-31   329.779541   356.100006   305.730011   331.536819   
       2018

#### 4. Build a crosstab with the earthquake data between the tsunami column and the magType column. Rather than showing the frequency count, show the maximum magnitude that was observed for each combination. Put the magnitude type along the columns.


In [827]:
quakes.head()

,mag,magType,time,place,tsunami,parsed_place
0,1.35,ml,1539475168010,"9km NE of Aguanga, CA",0,California
1,1.29,ml,1539475129610,"9km NE of Aguanga, CA",0,California
2,3.42,ml,1539475062610,"8km NE of Aguanga, CA",0,California
3,0.44,ml,1539474978070,"9km NE of Aguanga, CA",0,California
4,2.16,md,1539474716050,"10km NW of Avenal, CA",0,California


In [828]:
pd.crosstab(index=quakes.magType, columns=quakes.tsunami, values=quakes.mag, aggfunc=max )

tsunami,0,1
magType,,
mb,5.60,6.10
mb_lg,3.50,NaN
md,4.11,NaN
mh,1.10,NaN
ml,4.20,5.10
ms_20,NaN,5.70
mw,3.83,4.41
mwb,5.80,NaN
mwr,4.80,NaN


In [829]:
pd.crosstab(index=quakes.tsunami, columns= quakes.magType, values=quakes.mag, aggfunc=max )

magType,mb,mb_lg,md,mh,ml,ms_20,mw,mwb,mwr,mww
tsunami,,,,,,,,,,
0,5.6,3.5,4.11,1.1,4.2,NaN,3.83,5.8,4.8,6.0
1,6.1,NaN,NaN,NaN,5.1,5.7,4.41,NaN,NaN,7.5


In [830]:
quakes.query('magType=="mwb" and tsunami==0')['mag'].max()

5.8

#### 5. Calculate the rolling 60-day aggregations of the OHLC data by ticker for the FAANG data. Use the same aggregations as exercise 3.

In [831]:
( faang
 .groupby('ticker')
 .rolling('60D')
 .agg({
     'open':'mean',
     'high':'max',
     'low':'min',
     'close':'mean',
     'volume':'sum'})
)

open        high         low       close       volume
ticker date                                                                   
AAPL   2018-01-02   42.540001   43.075001   42.314999   43.064999  102223600.0
       2018-01-03   42.836250   43.637501   42.314999   43.061249  220295200.0
       2018-01-04   42.935833   43.637501   42.314999   43.126666  310033600.0
       2018-01-05   43.041875   43.842499   42.314999   43.282499  404673600.0
       2018-01-08   43.151000   43.902500   42.314999   43.343500  486944800.0
...                       ...         ...         ...         ...          ...
NFLX   2018-12-24  283.509251  332.049988  233.679993  281.931750  525657600.0
       2018-12-26  281.844501  332.049988  231.229996  280.777750  520444300.0
       2018-12-27  281.070489  332.049988  231.229996  280.162927  532679500.0
       2018-12-28  279.916342  332.049988  231.229996  279.461464  521973500.0
       2018-12-31  278.430770  332.049988  231.229996  277.451539  476314900.0

[1255 rows x 5 columns]

#### <div class="alert alert-info"><span style="color:green">    ***INTERESTING, CONSEQUENTIAL ERROR ON MY PART. I ORIGINALLY SPECIFIED `rolling(60)`, WHICH WILL DO THE LAST 60 OBSERVATIONS. WHICH IS A SPAN OF MORE THAN 60 DAYS WITH TRADING DATA. I SHOULD HAVE SPECIFIED `rolling('60D')`.*** </span></div>

 #### 6. Create a pivot table of the FAANG data that compares the stocks. Put the ticker in the rows and show the averages of the OHLC and volume traded data.

In [832]:
faang.head()

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-04,FB,186.210007,184.100006,184.899994,184.330002,13880900.0
2018-01-05,FB,186.899994,184.929993,185.589996,186.850006,13574500.0
2018-01-08,FB,188.899994,186.330002,187.199997,188.279999,17994700.0


In [833]:
pd.pivot_table(faang, index='ticker', aggfunc='mean', values=['open', 'high', 'low', 'close', 'volume'])

,close,high,low,open,volume
ticker,,,,,
AAPL,47.263357,47.748526,46.795877,47.277859,1.360803e+08
AMZN,1641.726176,1662.839839,1619.840519,1644.072709,5.648994e+06
FB,171.510956,173.613347,169.303148,171.472948,2.765860e+07
GOOG,1113.225134,1125.777606,1101.001658,1113.554101,1.741965e+06
NFLX,319.290319,325.219322,313.187330,319.620558,1.146962e+07


#### 7. Calculate the Z-scores for each numeric column of Amazon's data (ticker is AMZN) in Q4 2018 using `apply()`.

In [834]:
(
    faang
    .query('ticker=="AMZN"')
    .loc['2018-Q4']
    .drop(columns='ticker')
    .apply(
        lambda x: x.sub(x.mean()).div(x.std())
    )
)
    

,high,low,open,close,volume
date,,,,,
2018-10-01,2.368006,2.502113,2.337813,2.385848,-1.630411
2018-10-02,2.227302,2.247433,2.190795,2.155037,-0.861879
2018-10-03,2.058955,2.139987,2.068570,2.025489,-0.920345
2018-10-04,1.819474,1.781561,1.850048,1.722816,-0.126582
2018-10-05,1.628173,1.554416,1.642819,1.584748,-0.298771
...,...,...,...,...,...
2018-12-24,-2.159820,-2.187566,-2.179582,-2.226185,-0.141238
2018-12-26,-1.611714,-1.810493,-2.026617,-1.339674,1.123063
2018-12-27,-1.641276,-1.626703,-1.456521,-1.404343,0.849827


#### 8. Add event descriptions:   
a) Create a dataframe with the following three columns: ticker, date, and event. The columns should have the following values:    
> i) ticker: 'FB'    
ii) date: ['2018-07-25', '2018-03-19', '2018-03-20']    
iii) event: ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']

In [835]:
faang.head(3)

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-04,FB,186.210007,184.100006,184.899994,184.330002,13880900.0


In [836]:
event_df = pd.DataFrame({
    'ticker': 'FB',
    'date':  ['2018-07-25', '2018-03-19', '2018-03-20'],
    'event': ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']
    })

event_df.head()

,ticker,date,event
0,FB,2018-07-25,Disappointing user growth announced after close.
1,FB,2018-03-19,Cambridge Analytica story
2,FB,2018-03-20,FTC investigation


In [837]:
event_df.info() # first time through, I hadn't realized `date` wasn't a datetime in this dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ticker  3 non-null      object
 1   date    3 non-null      object
 2   event   3 non-null      object
dtypes: object(3)
memory usage: 200.0+ bytes


In [838]:
# make date datetime
event_df['date'] = pd.to_datetime(event_df['date'])
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ticker  3 non-null      object        
 1   date    3 non-null      datetime64[ns]
 2   event   3 non-null      object        
dtypes: datetime64[ns](1), object(2)
memory usage: 200.0+ bytes


In [839]:
# put date and ticker into the index
event_df = event_df.set_index(['date', 'ticker'])

In [840]:
event_df

,,event
date,ticker,
2018-07-25,FB,Disappointing user growth announced after close.
2018-03-19,FB,Cambridge Analytica story
2018-03-20,FB,FTC investigation


- event_df has a multi index
- faang has date only in the index
- Here's how to move `ticker` into faang's index w/out replacing `date', but rather augmenting it

In [841]:
faang.info() # faang's index IS datetime

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1255 entries, 2018-01-02 to 2018-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ticker  1255 non-null   object 
 1   high    1255 non-null   float64
 2   low     1255 non-null   float64
 3   open    1255 non-null   float64
 4   close   1255 non-null   float64
 5   volume  1255 non-null   float64
dtypes: float64(5), object(1)
memory usage: 68.6+ KB


In [842]:
# date was already in the index, reset_index kicks the existing index into the values
# to ADD ticker to the index, need to specify append=True

faang.set_index('ticker', append=True).sample(n=5)

,,high,low,open,close,volume
date,ticker,,,,,
2018-01-08,NFLX,212.500000,208.440002,210.020004,212.050003,5580200.0
2018-05-31,AMZN,1635.000000,1621.349976,1623.000000,1629.619995,3166300.0
2018-02-23,AMZN,1500.000000,1486.500000,1495.339966,1500.000000,4418100.0
2018-02-09,NFLX,255.800003,236.110001,253.850006,249.470001,16906900.0
2018-09-07,FB,164.630005,160.160004,160.309998,163.039993,24300600.0


In [843]:
faang_w_events= (
    faang
    .set_index('ticker', append=True) # add ticker to date in the index
    .merge(
        event_df, #needs to have date and ticker in the index...see above
        how='outer',
        left_index=True, # to merge on an index as opposed to a dataframe column
        right_index=True # ditto
    )
)

faang_w_events.head()

high          low         open        close  \
date       ticker                                                       
2018-01-02 AAPL      43.075001    42.314999    42.540001    43.064999   
           AMZN    1190.000000  1170.510010  1172.000000  1189.010010   
           FB       181.580002   177.550003   177.679993   181.419998   
           GOOG    1066.939941  1045.229980  1048.339966  1065.000000   
           NFLX     201.649994   195.419998   196.100006   201.070007   

                        volume event  
date       ticker                     
2018-01-02 AAPL    102223600.0   NaN  
           AMZN      2694500.0   NaN  
           FB       18151900.0   NaN  
           GOOG      1237600.0   NaN  
           NFLX     10966900.0   NaN

In [844]:
event_df.index

MultiIndex([('2018-07-25', 'FB'),
            ('2018-03-19', 'FB'),
            ('2018-03-20', 'FB')],
           names=['date', 'ticker'])

In [845]:
faang_w_events.loc[[x[0]  for x in event_df.index], :] #just picks out the dates to confirm they're in the merged dataframe

high          low         open        close  \
date       ticker                                                       
2018-07-25 AAPL      48.712502    48.107498    48.264999    48.705002   
           AMZN    1863.839966  1822.640015  1829.300049  1863.609985   
           FB       218.619995   214.270004   215.720001   217.500000   
           GOOG    1265.859985  1239.130005  1239.130005  1263.699951   
           NFLX     363.279999   355.649994   357.570007   362.869995   
2018-03-19 AAPL      44.367500    43.415001    44.330002    43.825001   
           AMZN    1561.660034  1525.349976  1554.530029  1544.930054   
           FB       177.169998   170.059998   177.009995   172.559998   
           GOOG    1121.989990  1089.010010  1120.010010  1099.819946   
           NFLX     317.000000   307.339996   315.799988   313.480011   
2018-03-20 AAPL      44.200001    43.735001    43.810001    43.810001   
           AMZN    1587.000000  1545.410034  1550.339966  1586.510010   
           FB       170.199997   161.949997   167.470001   168.149994   
           GOOG    1105.199951  1083.459961  1099.000000  1097.709961   
           NFLX     319.500000   312.799988   313.260010   317.500000   

                        volume  \
date       ticker                
2018-07-25 AAPL     66839600.0   
           AMZN      3738200.0   
           FB       58954200.0   
           GOOG      2127800.0   
           NFLX      8467800.0   
2018-03-19 AAPL    133787200.0   
           AMZN      6580800.0   
           FB       88140100.0   
           GOOG      2805900.0   
           NFLX      9925200.0   
2018-03-20 AAPL     78597600.0   
           AMZN      4581600.0   
           FB      129851800.0   
           GOOG      1831900.0   
           NFLX      5991900.0   

                                                              event  
date       ticker                                                    
2018-07-25 AAPL                                                 NaN  
           AMZN                                                 NaN  
           FB      Disappointing user growth announced after close.  
           GOOG                                                 NaN  
           NFLX                                                 NaN  
2018-03-19 AAPL                                                 NaN  
           AMZN                                                 NaN  
           FB                             Cambridge Analytica story  
           GOOG                                                 NaN  
           NFLX                                                 NaN  
2018-03-20 AAPL                                                 NaN  
           AMZN                                                 NaN  
           FB                                     FTC investigation  
           GOOG                                                 NaN  
           NFLX                                                 NaN

#### <span style="color:green"> 9. Use the `transform()` method on the FAANG data to represent all the values in terms of the first date in the data. To do so, divide all the values for each ticker by the values for the first date in the data for that ticker. This is referred to as an index, and the data for the first date is the base (https://ec.europa.eu/eurostat/statistics-explained/index.php/Beginners:Statistical_concept_-_Index_and_base_year). When data is in this format, we can easily see growth over time. Hint: transform() can take a function name.</span>

#### <div class="alert alert-info"><span style="color:green">    ***Very, very useful exercise. Worth reviewing in the future and documenting in Notion.*** </span></div>

In [846]:
faang.groupby('ticker').head(2)

,ticker,high,low,open,close,volume
date,,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,184.779999,181.330002,181.880005,184.669998,16886600.0
2018-01-02,AAPL,43.075001,42.314999,42.540001,43.064999,102223600.0
2018-01-03,AAPL,43.637501,42.990002,43.132500,43.057499,118071600.0
2018-01-02,AMZN,1190.000000,1170.510010,1172.000000,1189.010010,2694500.0
2018-01-03,AMZN,1205.489990,1188.300049,1188.300049,1204.199951,3108800.0
2018-01-02,NFLX,201.649994,195.419998,196.100006,201.070007,10966900.0
2018-01-03,NFLX,206.210007,201.500000,202.050003,205.050003,8591400.0
2018-01-02,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0


In [847]:
faang.shape

(1255, 6)

##### <span style="color:green" >To get the denominator for this calculation, we need to:</span>
##### <span style="color:green" >- Group by ticker</span>      
##### <span style="color:green" >- Find the first valid value for each ticker for each series</span>

##### <span style="color:green" >The first part isn't terribly difficult. Group by `ticker` and then get the `first` value for each series and use `transform()` to have it applied to each date for each ticker </span>

In [892]:
(
    faang
    .groupby('ticker')
    .transform('first')
    .loc[lambda x: x.index <= '2018-01-04'] # <<< Just so we can see the first few items of each groupby
)

,high,low,open,close,volume
date,,,,,
2018-01-02,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-04,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-02,43.075001,42.314999,42.540001,43.064999,102223600.0
2018-01-03,43.075001,42.314999,42.540001,43.064999,102223600.0
2018-01-04,43.075001,42.314999,42.540001,43.064999,102223600.0
2018-01-02,1190.000000,1170.510010,1172.000000,1189.010010,2694500.0
2018-01-03,1190.000000,1170.510010,1172.000000,1189.010010,2694500.0
2018-01-04,1190.000000,1170.510010,1172.000000,1189.010010,2694500.0


In [893]:
(
    faang
    .set_index('ticker', append=True)
    .groupby('ticker')
    .transform('first')
)

,,high,low,open,close,volume
date,ticker,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-04,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-05,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-08,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
...,...,...,...,...,...,...
2018-12-24,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0
2018-12-26,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0
2018-12-27,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0


In [850]:
# (
#     faang
#     .reset_index().set_index(['ticker', 'date'])
#     .groupby('ticker')
#     .transform('first')
# )

##### <span style="color:green" > When I saw this I had two thoughts:  </span>
##### <span style="color:green" >- This problem might not be that hard...divide all the values in `faang` by the values above...and we're done!</span>      
##### <span style="color:green" >- But....the `groupby` I did above has lost the tickers. Even if the calculations were correct, losing the ticker information makes the data close to useless in a practical sense and it also makes the accuracy difficult to confirm. </span>

##### <span style="color:green" > Still...I went ahead and performed the calculation to make sure the general method would work. I had to drop the `ticker` column to make the columns lined up, but this *looks* like the calculation you'd want.   </span>

In [851]:
faang.drop(columns='ticker').div(faang.groupby('ticker').transform('first'))

,high,low,open,close,volume
date,,,,,
2018-01-02,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-03,1.017623,1.021290,1.023638,1.017914,0.930294
2018-01-04,1.025498,1.036891,1.040635,1.016040,0.764708
2018-01-05,1.029298,1.041566,1.044518,1.029931,0.747828
2018-01-08,1.040313,1.049451,1.053579,1.037813,0.991340
...,...,...,...,...,...
2018-12-24,0.940578,0.928131,0.928993,0.916638,1.284987
2018-12-26,0.974750,0.940463,0.943406,0.976019,1.917663
2018-12-27,0.978396,0.953857,0.970248,0.980169,1.704751


##### <span style="color:green" > To make this work, before doing any of the grouping and transforming...I needed to move the ticker into index along w/the date...</span>     
##### <span style="color:green" > ...`set_index` to `ticker` would drop `date` and replace it with `ticker`. So I had to find the `append=True` option for `set_index` to put both `date` and `ticker` in the index.</span>     

In [894]:
faang.set_index('ticker', append=True).sample(10)

,,high,low,open,close,volume
date,ticker,,,,,
2018-07-25,GOOG,1265.859985,1239.130005,1239.130005,1263.699951,2127800.0
2018-03-23,AMZN,1549.020020,1495.359985,1539.010010,1495.560059,8006000.0
2018-08-08,FB,186.850006,183.759995,184.750000,185.179993,22205200.0
2018-06-12,GOOG,1139.790039,1130.734985,1131.069946,1139.319946,912000.0
2018-11-21,AMZN,1550.000000,1515.000000,1542.989990,1516.729980,5716800.0
2018-01-29,FB,188.839996,185.630005,188.750000,185.979996,20453200.0
2018-12-06,FB,139.699997,133.669998,133.820007,139.630005,28218100.0
2018-11-19,NFLX,285.089996,269.149994,283.790009,270.600006,12993800.0
2018-02-02,NFLX,270.619995,262.709991,263.000000,267.429993,9123600.0


#### <span style="color:green" > Repeating what I did above, but with `date` and `ticker` in the index was straightforward.</span>    

In [895]:
(
    faang
    .set_index('ticker', append=True)
    .apply(
        lambda x: x.groupby(['ticker']).transform('first')
    )
)
        
        

,,high,low,open,close,volume
date,ticker,,,,,
2018-01-02,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-03,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-04,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-05,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
2018-01-08,FB,181.580002,177.550003,177.679993,181.419998,18151900.0
...,...,...,...,...,...,...
2018-12-24,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0
2018-12-26,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0
2018-12-27,GOOG,1066.939941,1045.229980,1048.339966,1065.000000,1237600.0


##### <span style="color:green" > Finally, I put it all together:</span>    
##### <span style="color:green" > - The numerator is just `faang` with both `date` and `ticker` in index.</span>    
##### <span style="color:green" > - The denominators are the first values for each ticker/series that we created above, again with both `date` and `ticker` in the index.</span>    

In [896]:
faang_indexed = \
(faang
 .set_index('ticker', append=True) # put both `date` and `ticker` into faang's index
 .div(faang                                  # divide by the first value of each ticker, using what we did in the cell above
      .set_index('ticker', append=True)
      .apply(lambda x: x.groupby(['ticker']).transform('first'))
     )
)

faang_indexed.head()


,,high,low,open,close,volume
date,ticker,,,,,
2018-01-02,FB,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-03,FB,1.017623,1.021290,1.023638,1.017914,0.930294
2018-01-04,FB,1.025498,1.036891,1.040635,1.016040,0.764708
2018-01-05,FB,1.029298,1.041566,1.044518,1.029931,0.747828
2018-01-08,FB,1.040313,1.049451,1.053579,1.037813,0.991340


##### <span style="color:green" > (After reviewing things, a much more intuitive solution is below:)</span>  

In [897]:
(faang
 .set_index('ticker', append=True) # creates the multi-index w/date and ticker in the index as for the 'numerator' part of the calculation
 .div(
     faang
    .set_index('ticker', append=True) # puts date and ticker in the index as for the 'denominator' part of the calculation
    .groupby('ticker')
    .transform('first')
 )
).query('date <= "2018-01-04"')


,,high,low,open,close,volume
date,ticker,,,,,
2018-01-02,FB,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-03,FB,1.017623,1.021290,1.023638,1.017914,0.930294
2018-01-04,FB,1.025498,1.036891,1.040635,1.016040,0.764708
2018-01-02,AAPL,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-03,AAPL,1.013059,1.015952,1.013928,0.999826,1.155033
2018-01-04,AAPL,1.006790,1.016661,1.013987,1.004470,0.877864
2018-01-02,AMZN,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-03,AMZN,1.013017,1.015199,1.013908,1.012775,1.153758
2018-01-04,AMZN,1.021739,1.029175,1.028157,1.017308,1.121581


#### <span style="color:green" > So I did it! </span>  
#### <span style="color:green" > But I also needed to confirm it worked.  </span>    
#### <span style="color:green" > One thing that needed to be true if this solution was correct was that the first date for each ticker for each series would be equal to 1.0000. </span> 
#### <span style="color:green" > Doing this with regular indexing wasn't very easy. What I started with is further down.  </span>  
#### <span style="color:green" > However, doing this with `query` turns out to be blessedly easy and straightforward. See the code and output below. </span>  
#### <span style="color:green" > ALSO NOTE THAT `first` PROBABLY DEPENDS ON THE DATA BEING SORTED BY `date`.  </span>  

#### <span style="color:green" > `query()` makes it very easy to select rows, in the index or from the columns.  </span>  
#### <span style="color:green" > (Apparently, selecting columns with `query()` is strongly discouraged.)  </span>  

### <span style="color:green" > **Pick out single values of `date` and `ticker`.** </span>  

In [856]:
faang_indexed.loc["12-17-18", "AMZN"]

high      1.324479
low       1.285773
open      1.336177
close     1.279140
volume    3.276972
Name: (2018-12-17 00:00:00, AMZN), dtype: float64

#### <span style="color:green" > This works as well. The whole query clause has to be placed in quotes and each conditional needs to be placed in parentheses () in order to avoid confusion on the part of python regarding operator precedence.  </span>  

In [857]:
faang_indexed.query('(date == "12-17-18") & (ticker == "AMZN")')

,,high,low,open,close,volume
date,ticker,,,,,
2018-12-17,AMZN,1.324479,1.285773,1.336177,1.27914,3.276972


### <span style="color:green" > **Pick out multiple values of `date` and `ticker`.** </span>  

##### <span style="color:green" > Key details: </span>  
##### <span style="color:green" > - The index choices must be enclosed in parentheses </span>  
##### <span style="color:green" > - The indexing for both the index and the columns MUST be included </span>  
##### <span style="color:green" > - In this example, that's why `(("12-17-18", "12-21-18"), ("AMZN", "GOOG"))` is followed by `, :`. The latter is the specification of "all columns" </span>  

In [858]:
faang_indexed.loc[(("12-17-18", "12-21-18"), ("AMZN", "GOOG")), :]

high       low      open     close    volume
date       ticker                                                  
2018-12-17 AMZN    1.324479  1.285773  1.336177  1.279140  3.276972
           GOOG    0.987075  0.964285  0.989669  0.954488  1.927440
2018-12-21 AMZN    1.243697  1.165270  1.249991  1.158485  5.062275
           GOOG    0.959773  0.931556  0.968484  0.919756  3.713639

##### <span style="color:green" > A *nearly* equivalent formulation with `query()`.</span>  

In [859]:
faang_indexed.query('(date in  ["12-17-18", "12-21-18"]) & (ticker in ["AMZN", "GOOG"])')

,,high,low,open,close,volume
date,ticker,,,,,
2018-12-17,AMZN,1.324479,1.285773,1.336177,1.279140,3.276972
2018-12-21,AMZN,1.243697,1.165270,1.249991,1.158485,5.062275
2018-12-17,GOOG,0.987075,0.964285,0.989669,0.954488,1.927440
2018-12-21,GOOG,0.959773,0.931556,0.968484,0.919756,3.713639


##### <span style="color:green" > Adding `sort_index` addresses the difference.</span> 

In [860]:
faang_indexed.query('(date in  ["12-17-18", "12-21-18"]) & (ticker in ["AMZN", "GOOG"])').sort_index(level=0, axis=0)

high       low      open     close    volume
date       ticker                                                  
2018-12-17 AMZN    1.324479  1.285773  1.336177  1.279140  3.276972
           GOOG    0.987075  0.964285  0.989669  0.954488  1.927440
2018-12-21 AMZN    1.243697  1.165270  1.249991  1.158485  5.062275
           GOOG    0.959773  0.931556  0.968484  0.919756  3.713639

### <span style="color:green" > **Pick out multiple `date` values but ALL `ticker`s.** </span>  

In [861]:
faang_indexed.loc[(("12-17-18", "12-19-18"), slice(None)), :]

high       low      open     close    volume
date       ticker                                                  
2018-12-17 FB      0.798106  0.779611  0.805268  0.772737  1.340576
           AAPL    0.977075  0.961420  0.972320  0.951701  1.732981
           AMZN    1.324479  1.285773  1.336177  1.279140  3.276972
           NFLX    1.353732  1.335994  1.359052  1.307007  0.878525
           GOOG    0.987075  0.964285  0.989669  0.954488  1.927440
2018-12-19 FB      0.798050  0.746269  0.794743  0.734428  3.162473
           AAPL    0.971851  0.939915  0.975552  0.933995  1.919216
           AMZN    1.331538  1.267123  1.316596  1.257416  3.263017
           NFLX    1.392859  1.349759  1.376644  1.326752  1.257274
           GOOG    0.995370  0.964429  0.986312  0.960573  2.003313

##### <span style="color:green" > `IndexSlice` allows you to stop using `Slice` and parentheses and return to using the more natural `:` and `[]`s .</span> 

In [862]:
idx = pd.IndexSlice
faang_indexed.loc[ idx[["12-17-18", "12-19-18"], :], idx[:]]

high       low      open     close    volume
date       ticker                                                  
2018-12-17 FB      0.798106  0.779611  0.805268  0.772737  1.340576
           AAPL    0.977075  0.961420  0.972320  0.951701  1.732981
           AMZN    1.324479  1.285773  1.336177  1.279140  3.276972
           NFLX    1.353732  1.335994  1.359052  1.307007  0.878525
           GOOG    0.987075  0.964285  0.989669  0.954488  1.927440
2018-12-19 FB      0.798050  0.746269  0.794743  0.734428  3.162473
           AAPL    0.971851  0.939915  0.975552  0.933995  1.919216
           AMZN    1.331538  1.267123  1.316596  1.257416  3.263017
           NFLX    1.392859  1.349759  1.376644  1.326752  1.257274
           GOOG    0.995370  0.964429  0.986312  0.960573  2.003313

In [863]:
faang_indexed.loc[ idx[["12-17-18", "12-19-18"], :], idx['high':'open']]

high       low      open
date       ticker                              
2018-12-17 FB      0.798106  0.779611  0.805268
           AAPL    0.977075  0.961420  0.972320
           AMZN    1.324479  1.285773  1.336177
           NFLX    1.353732  1.335994  1.359052
           GOOG    0.987075  0.964285  0.989669
2018-12-19 FB      0.798050  0.746269  0.794743
           AAPL    0.971851  0.939915  0.975552
           AMZN    1.331538  1.267123  1.316596
           NFLX    1.392859  1.349759  1.376644
           GOOG    0.995370  0.964429  0.986312

##### <span style="color:green" > The `query` version of this is probably the easiest of them all, but would require sorting the index to produce the same ordering at the other two solutions: </span> 

In [864]:

faang_indexed.query('date in("12-17-18", "12-19-18")')

,,high,low,open,close,volume
date,ticker,,,,,
2018-12-17,FB,0.798106,0.779611,0.805268,0.772737,1.340576
2018-12-19,FB,0.798050,0.746269,0.794743,0.734428,3.162473
2018-12-17,AAPL,0.977075,0.961420,0.972320,0.951701,1.732981
2018-12-19,AAPL,0.971851,0.939915,0.975552,0.933995,1.919216
2018-12-17,AMZN,1.324479,1.285773,1.336177,1.279140,3.276972
2018-12-19,AMZN,1.331538,1.267123,1.316596,1.257416,3.263017
2018-12-17,NFLX,1.353732,1.335994,1.359052,1.307007,0.878525
2018-12-19,NFLX,1.392859,1.349759,1.376644,1.326752,1.257274
2018-12-17,GOOG,0.987075,0.964285,0.989669,0.954488,1.927440


### <span style="color:green" > **Pick out a `date` range.** </span>  

##### <span style="color:green" > Interestingly...this will NOT work: </span> 

In [888]:
#faang_indexed.loc[(slice("2018-01","2018-03"), slice(None)), :]

##### <span style="color:green" > `slice`ing on dates requires the index to be sorted </span> 

In [866]:
faang_indexed.sort_values(by=['date', 'ticker'], inplace=True)
faang_indexed.loc[(slice("2018-01","2018-03"), slice(None)), :]

high       low      open     close    volume
date       ticker                                                  
2018-01-02 AAPL    1.000000  1.000000  1.000000  1.000000  1.000000
           AMZN    1.000000  1.000000  1.000000  1.000000  1.000000
           FB      1.000000  1.000000  1.000000  1.000000  1.000000
           GOOG    1.000000  1.000000  1.000000  1.000000  1.000000
           NFLX    1.000000  1.000000  1.000000  1.000000  1.000000
...                     ...       ...       ...       ...       ...
2018-03-29 AAPL    0.996808  0.986057  0.986189  0.973993  1.502530
           AMZN    1.223084  1.166329  1.199659  1.217265  4.669178
           FB      0.888975  0.868150  0.873199  0.880774  3.274274
           GOOG    0.977562  0.959502  0.964983  0.968817  2.203297
           NFLX    1.464667  1.411831  1.463539  1.468891  1.745753

[305 rows x 5 columns]

##### <span style="color:green" > IndexSlicer again lets us use clearer syntax, or at least similar to other Pandas and Python syntax </span> 

In [867]:
faang_indexed.loc[idx["2018-01" : "2018-03", :],  idx[:]]

high       low      open     close    volume
date       ticker                                                  
2018-01-02 AAPL    1.000000  1.000000  1.000000  1.000000  1.000000
           AMZN    1.000000  1.000000  1.000000  1.000000  1.000000
           FB      1.000000  1.000000  1.000000  1.000000  1.000000
           GOOG    1.000000  1.000000  1.000000  1.000000  1.000000
           NFLX    1.000000  1.000000  1.000000  1.000000  1.000000
...                     ...       ...       ...       ...       ...
2018-03-29 AAPL    0.996808  0.986057  0.986189  0.973993  1.502530
           AMZN    1.223084  1.166329  1.199659  1.217265  4.669178
           FB      0.888975  0.868150  0.873199  0.880774  3.274274
           GOOG    0.977562  0.959502  0.964983  0.968817  2.203297
           NFLX    1.464667  1.411831  1.463539  1.468891  1.745753

[305 rows x 5 columns]

##### <span style="color:green" > This might be a case where `query()` isn't as handy as it was in other applications. There may be a way to get it to handle date shorthand like "2018-Q1" or  "2018-01":"2018-03", but I couldn't find a good reference for how to do it. </span> 

In [868]:
start_date = '2018-01-01'
end_date = '2018-03-31'
faang_indexed.query('(date >= @start_date) & (date <= @end_date)')

high       low      open     close    volume
date       ticker                                                  
2018-01-02 AAPL    1.000000  1.000000  1.000000  1.000000  1.000000
           AMZN    1.000000  1.000000  1.000000  1.000000  1.000000
           FB      1.000000  1.000000  1.000000  1.000000  1.000000
           GOOG    1.000000  1.000000  1.000000  1.000000  1.000000
           NFLX    1.000000  1.000000  1.000000  1.000000  1.000000
...                     ...       ...       ...       ...       ...
2018-03-29 AAPL    0.996808  0.986057  0.986189  0.973993  1.502530
           AMZN    1.223084  1.166329  1.199659  1.217265  4.669178
           FB      0.888975  0.868150  0.873199  0.880774  3.274274
           GOOG    0.977562  0.959502  0.964983  0.968817  2.203297
           NFLX    1.464667  1.411831  1.463539  1.468891  1.745753

[305 rows x 5 columns]

#### 10. The European Centre for Disease Prevention and Control (ECDC) provides an open dataset on COVID-19 cases called daily number of new reported cases of COVID-19 by country worldwide (https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide). This dataset is updated daily, but we will use a snapshot that contains data through September 18, 2020. Complete the following tasks to practice the skills you've learned up to this point in the book: 

a) Prepare the data:   
i) Read in the data in the covid19_cases.csv file.   
ii) Create a date column by parsing the dateRep column into a datetime.    
iii) Set the date column as the index.     
iv) Use the replace() method to update all occurrences of United_States_of_America and United_Kingdom to USA and UK, respectively.      
v) Sort the index.     
b) For the five countries with the most cases (cumulative), find the day with the largest number of cases.


In [869]:
!head -5 ~/Downloads/data.csv

dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757,Asia,9.01377925
13/12/2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757,Asia,7.05277624
12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757,Asia,6.86876792
11/12/2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757,Asia,7.13426564


In [870]:
covid = pd.read_csv('~/Downloads/data.csv', index_col='dateRep', parse_dates=True, dayfirst=True)
covid.head()

,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
dateRep,,,,,,,,,,,
2020-12-14,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.013779
2020-12-13,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.052776
2020-12-12,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
2020-12-11,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.134266
2020-12-10,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.968658


In [898]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43518 entries, 2020-09-18 to 2020-03-21
Data columns (total 11 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   day                                                         43518 non-null  int64  
 1   month                                                       43518 non-null  int64  
 2   year                                                        43518 non-null  int64  
 3   cases                                                       43518 non-null  int64  
 4   deaths                                                      43518 non-null  int64  
 5   countriesAndTerritories                                     43518 non-null  object 
 6   geoId                                                       43330 non-null  object 
 7   countryterritoryCode                                        43454 no

In [872]:
#?pd.read_csv

In [873]:
#??pd.DataFrame.replace

#### The instructions say the data is through 9/18/2020, so we'll cap the new data at that date so my results can match hers

In [874]:
covid.shape

(61900, 11)

In [875]:
covid = covid.query('dateRep <= "2020-09-18"')
covid.shape

(43518, 11)

In [876]:
covid.query('countriesAndTerritories in ["United_States_of_America", "United_Kingdom"]').sample(8)

,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
dateRep,,,,,,,,,,,
2020-06-08,8,6,2020,22302,712,United_States_of_America,US,USA,329064917.0,America,90.901517
2020-06-27,27,6,2020,45527,623,United_States_of_America,US,USA,329064917.0,America,127.285219
2020-08-13,13,8,2020,55941,1490,United_States_of_America,US,USA,329064917.0,America,234.046828
2020-09-08,8,9,2020,24250,267,United_States_of_America,US,USA,329064917.0,America,170.106861
2020-05-31,31,5,2020,23297,945,United_States_of_America,US,USA,329064917.0,America,91.927150
2020-08-07,7,8,2020,950,18,United_Kingdom,UK,GBR,66647112.0,Europe,14.557270
2020-01-17,17,1,2020,0,0,United_States_of_America,US,USA,329064917.0,America,0.000000
2020-05-07,7,5,2020,24128,2353,United_States_of_America,US,USA,329064917.0,America,117.294181


In [877]:
covid_u = covid.replace({'United_States_of_America':'USA', 'United_Kingdom':'UK'})

In [878]:
covid_u.query('countriesAndTerritories in ["United_States_of_America", "United_Kingdom"]')

,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
dateRep,,,,,,,,,,,


In [879]:
covid_u.sort_index(inplace=True)

In [880]:
covid_u.head()

,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
dateRep,,,,,,,,,,,
2019-12-31,31,12,2019,0,0,Luxembourg,LU,LUX,613894.0,Europe,NaN
2019-12-31,31,12,2019,0,0,Sweden,SE,SWE,10230185.0,Europe,NaN
2019-12-31,31,12,2019,0,0,Armenia,AM,ARM,2957728.0,Europe,NaN
2019-12-31,31,12,2019,0,0,Romania,RO,ROU,19414458.0,Europe,NaN
2019-12-31,31,12,2019,0,0,Belgium,BE,BEL,11455519.0,Europe,NaN


In [881]:
covid_u.tail()

,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
dateRep,,,,,,,,,,,
2020-09-18,18,9,2020,0,0,Dominica,DM,DMA,71808.0,America,5.570410
2020-09-18,18,9,2020,0,0,United_States_Virgin_Islands,VI,VIR,104579.0,America,84.146913
2020-09-18,18,9,2020,359,9,Greece,EL,GRC,10724599.0,Europe,31.721466
2020-09-18,18,9,2020,98,0,Slovenia,SI,SVN,2080908.0,Europe,48.872896
2020-09-18,18,9,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,1.535155


In [882]:
covid_u.index.max()

Timestamp('2020-09-18 00:00:00')

#### 10b. For the five countries with the most cases (cumulative), find the day with the largest number of cases.

In [899]:
covid_5=\
(covid
 .assign(
     
     tot_cases = lambda x: x.groupby('countriesAndTerritories').cases.transform('sum'),
     case_rank = lambda x: x.tot_cases.rank(method='dense', ascending=False)
     
 ).query('case_rank <= 5')
)

In [900]:
covid_5.groupby('countriesAndTerritories').cases.max()

countriesAndTerritories
Brazil                      69074
India                       97894
Peru                        10143
Russia                      11656
United_States_of_America    78427
Name: cases, dtype: int64

In [901]:
covid_5.groupby('countriesAndTerritories').cases.idxmax()

countriesAndTerritories
Brazil                     2020-07-30
India                      2020-09-17
Peru                       2020-08-17
Russia                     2020-05-12
United_States_of_America   2020-07-25
Name: cases, dtype: datetime64[ns]

In [902]:
covid_5.groupby('countriesAndTerritories').cases.idxmax()

countriesAndTerritories
Brazil                     2020-07-30
India                      2020-09-17
Peru                       2020-08-17
Russia                     2020-05-12
United_States_of_America   2020-07-25
Name: cases, dtype: datetime64[ns]

In [903]:
covid.groupby('countriesAndTerritories').cases.sum().nlargest(5)

countriesAndTerritories
United_States_of_America    6674458
India                       5214677
Brazil                      4455386
Russia                      1085281
Peru                         750098
Name: cases, dtype: int64